# Forecasting Optuna Search CV 
## Modulus Set 3

**Notebook Goal**
- A modeling pipeline that optimizes the hyperparameters of the sktime forecasters that have the [capavility:pred_int tag](https://www.sktime.net/en/stable/examples/01b_forecasting_proba.html) 
- This notebook will focus on the ones where `i mod 4 = 3` wher `i` is the index of the registry table in the above link.
- The work will be based on this documentation: [ForecastingOptunaSearchCV](https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ForecastingOptunaSearchCV.html)

In [29]:
from sktime.registry import all_estimators
from src.data.data_loader import load_data
from src.data.data_cleaner import clean_data

In [28]:
# Load the autoreload extension
%load_ext autoreload
# Reload all modules automatically before executing code
%autoreload 2

In [30]:
df = load_data()
# Get the list of forecasters with prediction interval capability

In [ ]:
# Load models (which is gonna take a while)
models = all_estimators(
    "forecaster", filter_tags={"capability:pred_int": True}, as_dataframe=True
)

In [31]:
df = clean_data(df)
print(df.columns)

Index(['id', 'p_num', 'time', 'bg-0:00', 'insulin-0:00', 'carbs-0:00',
       'hr-0:00', 'steps-0:00', 'cals-0:00', 'activity-0:00'],
      dtype='object')


In [100]:
filtered_models = models.iloc[3::4]

## Examples of each patient's data:

### Patient 1

In [101]:
df[df["p_num"] == "p01"].head()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
0,p01_0,p01,06:10:00,15.1,0.0417,NaN,NaN,NaN,NaN,NaN
1,p01_1,p01,06:25:00,14.4,0.0417,NaN,NaN,NaN,NaN,NaN
2,p01_2,p01,06:40:00,13.9,0.0417,NaN,NaN,NaN,NaN,NaN
3,p01_3,p01,06:55:00,13.8,0.0417,NaN,NaN,NaN,NaN,NaN
4,p01_4,p01,07:10:00,13.4,0.0417,NaN,NaN,NaN,NaN,NaN


In [102]:
df[df["p_num"] == "p01"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,7716.000000,8459.000000,149.000000,8017.000000,5895.000000,8384.000000
mean,8.917146,0.130766,24.469799,77.285007,76.119932,9.029529
std,4.130287,0.695182,13.245885,16.679275,130.683944,6.572246
min,2.200000,0.000000,4.000000,46.600000,0.000000,4.800000
25%,5.600000,0.029200,15.000000,62.600000,0.000000,4.800000
50%,8.200000,0.041700,20.000000,76.000000,18.000000,5.495000
75%,11.600000,0.058300,30.000000,88.100000,89.000000,10.980000
max,27.800000,11.741700,80.000000,172.400000,741.000000,53.000000


### Patient 2

In [103]:
df[df["p_num"] == "p02"].head()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
8459,p02_0,p02,06:05:00,6.7,0.0510,NaN,NaN,NaN,NaN,NaN
8460,p02_1,p02,06:10:00,6.5,0.0567,NaN,NaN,NaN,NaN,NaN
8461,p02_2,p02,06:15:00,6.5,0.0583,NaN,NaN,NaN,NaN,NaN
8462,p02_3,p02,06:20:00,6.5,0.0573,NaN,NaN,NaN,NaN,NaN
8463,p02_4,p02,06:25:00,6.5,0.0580,NaN,NaN,NaN,NaN,NaN


In [104]:
df[df["p_num"] == "p02"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,25757.000000,25872.000000,329.000000,5594.000000,4155.000000,7494.000000
mean,9.336584,0.180665,53.337386,83.364587,132.211793,4.684996
std,2.928298,0.690833,42.650078,17.968259,178.699497,6.941970
min,2.200000,0.000000,10.000000,40.000000,1.000000,0.030000
25%,7.200000,0.050000,32.000000,70.000000,31.000000,1.660000
50%,8.800000,0.080250,50.000000,80.000000,75.000000,3.000000
75%,11.000000,0.100000,60.000000,93.100000,154.000000,5.400000
max,22.200000,18.083300,660.000000,185.300000,1359.000000,116.100000


### Patient 3

In [105]:
df[df["p_num"] == "p03"].head()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
34331,p03_0,p03,06:05:00,7.6,0.0833,NaN,NaN,NaN,NaN,NaN
34332,p03_1,p03,06:10:00,7.3,0.1100,NaN,NaN,NaN,NaN,NaN
34333,p03_2,p03,06:15:00,7.2,0.1252,NaN,NaN,NaN,NaN,NaN
34334,p03_3,p03,06:20:00,7.2,0.1188,NaN,NaN,NaN,NaN,NaN
34335,p03_4,p03,06:25:00,7.1,0.1167,NaN,NaN,NaN,NaN,NaN


In [106]:
df[df["p_num"] == "p03"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,25930.000000,26028.000000,306.000000,23329.000000,15188.000000,24649.000000
mean,8.588785,0.270292,82.163399,75.818029,21.644785,8.422431
std,3.144037,1.194460,64.072291,16.810880,65.700752,7.020719
min,2.200000,0.000000,20.000000,47.200000,0.000000,1.120000
25%,6.300000,0.083300,60.000000,63.000000,0.000000,5.600000
50%,7.900000,0.132200,80.000000,73.600000,0.000000,5.840000
75%,10.400000,0.191700,90.000000,83.900000,11.000000,7.300000
max,22.200000,46.311000,852.000000,156.800000,582.000000,65.850000


### Patient 4

In [107]:
df[df["p_num"] == "p04"].head()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
60359,p04_0,p04,06:05:00,6.7,0.0706,NaN,NaN,NaN,NaN,NaN
60360,p04_1,p04,06:10:00,6.6,0.0901,NaN,NaN,NaN,NaN,NaN
60361,p04_2,p04,06:15:00,6.2,0.0756,NaN,NaN,NaN,NaN,NaN
60362,p04_3,p04,06:20:00,6.0,0.0482,NaN,NaN,NaN,NaN,NaN
60363,p04_4,p04,06:25:00,5.7,0.0205,NaN,NaN,NaN,NaN,NaN


In [108]:
df[df["p_num"] == "p04"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,24532.000000,24686.000000,462.000000,12535.000000,9210.000000,21898.000000
mean,7.765555,0.151833,29.181818,82.841245,46.061889,6.455725
std,2.246324,0.685633,25.883048,16.680994,94.340476,4.235032
min,2.200000,0.000000,6.000000,50.700000,0.000000,0.910000
25%,6.200000,0.045500,15.000000,70.500000,0.000000,4.550000
50%,7.400000,0.062500,24.000000,82.000000,12.000000,4.550000
75%,9.000000,0.070800,35.000000,92.400000,46.000000,6.180000
max,18.400000,42.780000,332.000000,164.400000,673.000000,42.510000


### Patient 5

In [109]:
df[df["p_num"] == "p05"].head()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
85045,p05_0,p05,06:05:00,3.6,0.0729,NaN,NaN,NaN,NaN,NaN
85046,p05_1,p05,06:20:00,3.7,0.0729,NaN,NaN,NaN,NaN,NaN
85047,p05_2,p05,06:35:00,3.8,0.0729,NaN,NaN,NaN,NaN,NaN
85048,p05_3,p05,06:50:00,3.9,0.0729,NaN,NaN,NaN,NaN,NaN
85049,p05_4,p05,07:05:00,4.0,0.0729,NaN,NaN,NaN,NaN,NaN


In [110]:
df[df["p_num"] == "p05"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,7591.000000,8288.000000,141.000000,6911.000000,3489.000000,6365.000000
mean,8.142485,0.152484,25.765957,74.409044,38.283749,6.855614
std,3.128485,0.568694,13.195583,15.284031,84.710880,4.479553
min,2.200000,0.000000,2.000000,43.700000,0.000000,4.700000
25%,5.700000,0.066700,16.000000,62.900000,0.000000,4.750000
50%,7.800000,0.072900,25.000000,74.600000,6.000000,4.850000
75%,10.100000,0.079200,35.000000,84.200000,38.000000,6.670000
max,22.200000,8.154200,85.000000,153.700000,660.000000,45.380000


### Patient 6

In [111]:
df[df["p_num"] == "p06"].head()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
93333,p06_0,p06,10:25:00,11.4,0.0833,NaN,NaN,NaN,NaN,NaN
93334,p06_1,p06,10:40:00,11.3,0.0833,NaN,NaN,NaN,NaN,NaN
93335,p06_2,p06,10:55:00,10.9,0.0833,NaN,NaN,NaN,NaN,NaN
93336,p06_3,p06,11:10:00,10.4,0.0833,NaN,NaN,NaN,NaN,NaN
93337,p06_4,p06,11:25:00,10.4,0.0833,NaN,NaN,NaN,NaN,NaN


In [112]:
df[df["p_num"] == "p06"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,7845.000000,8383.000000,80.000000,6746.000000,5325.000000,7984.000000
mean,8.953499,0.226139,58.075000,72.878106,62.233615,11.470765
std,3.806012,1.020254,24.132427,17.816539,114.967930,8.611394
min,2.900000,0.000000,7.000000,39.000000,0.000000,6.150000
25%,6.200000,0.083300,40.000000,57.400000,0.000000,6.150000
50%,8.100000,0.083300,54.500000,72.400000,8.000000,7.520000
75%,10.900000,0.087500,76.000000,85.100000,67.000000,13.190000
max,27.800000,14.183300,130.000000,152.800000,627.000000,68.030000


### Patient 10

In [113]:
df[df["p_num"] == "p10"].tail()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
127165,p10_25449,p10,19:55:00,6.9,0.0315,NaN,85.7,23.0,12.92,NaN
127166,p10_25450,p10,20:00:00,6.7,0.0276,NaN,87.4,168.0,9.64,NaN
127167,p10_25451,p10,20:05:00,6.4,0.0243,NaN,82.1,0.0,7.53,NaN
127168,p10_25452,p10,20:10:00,6.4,0.0168,NaN,93.6,0.0,7.65,NaN
127169,p10_25453,p10,20:15:00,6.5,0.0232,NaN,94.1,77.0,9.16,NaN


In [114]:
df[df["p_num"] == "p10"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,25324.000000,25454.000000,356.000000,23576.000000,13595.000000,19260.000000
mean,6.373914,0.096138,51.106742,78.180493,60.403384,10.890315
std,1.579262,0.385269,27.145510,17.951637,128.808696,8.570162
min,2.200000,0.000000,10.000000,47.400000,0.000000,1.410000
25%,5.300000,0.014700,30.000000,62.900000,0.000000,5.850000
50%,6.000000,0.055450,50.000000,77.500000,8.000000,7.280000
75%,7.200000,0.073075,65.000000,88.500000,50.000000,11.990000
max,15.900000,9.083300,170.000000,184.200000,783.000000,71.770000


### Patient 11

Approx 2038 hours of data (24466*5min)/60min, -> 85 days.
- 5 minute interval

In [115]:
df[df["p_num"] == "p11"].head()

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
127170,p11_0,p11,06:05:00,9.3,NaN,NaN,NaN,NaN,NaN,NaN
127171,p11_1,p11,06:10:00,9.2,NaN,NaN,NaN,NaN,NaN,NaN
127172,p11_2,p11,06:15:00,9.1,NaN,NaN,NaN,NaN,NaN,NaN
127173,p11_3,p11,06:20:00,9.1,NaN,NaN,NaN,NaN,NaN,NaN
127174,p11_4,p11,06:25:00,9.2,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
df[df["p_num"] == "p11"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,24466.000000,15180.000000,188.000000,15360.000000,10007.00000,23165.000000
mean,9.376024,0.166984,38.425532,77.786335,56.05686,8.511119
std,2.882044,0.528628,17.121765,17.152982,112.22407,6.015125
min,2.200000,0.000000,3.000000,49.700000,0.00000,1.190000
25%,7.300000,0.075000,25.000000,63.300000,0.00000,5.950000
50%,9.200000,0.108300,38.000000,76.000000,7.00000,5.950000
75%,11.300000,0.108300,50.000000,87.800000,53.00000,7.620000
max,21.600000,12.416700,95.000000,164.800000,660.00000,66.390000


### Patient 12
Approx 2097 hours of data (25167*5min)/60min, -> 90 days.
- 5 minute interval

In [117]:
df[df["p_num"] == "p12"].iloc[0 : 12 * 6]

,id,p_num,time,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00,activity-0:00
151725,p12_0,p12,10:25:00,4.1,0.0,NaN,63.9,NaN,NaN,NaN
151726,p12_1,p12,10:30:00,4.2,0.0,NaN,89.3,NaN,NaN,NaN
151727,p12_2,p12,10:35:00,4.3,0.0,NaN,112.6,NaN,NaN,NaN
151728,p12_3,p12,10:40:00,4.5,0.0,NaN,105.5,NaN,NaN,NaN
151729,p12_4,p12,10:45:00,4.1,0.0,NaN,106.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
151792,p12_67,p12,16:15:00,7.3,0.0,NaN,118.0,NaN,NaN,NaN
151793,p12_68,p12,16:20:00,7.1,0.0,NaN,116.7,NaN,NaN,NaN
151794,p12_69,p12,16:25:00,6.7,0.0,NaN,114.3,NaN,NaN,NaN
151795,p12_70,p12,16:30:00,6.4,0.0,NaN,114.4,NaN,NaN,NaN


In [118]:
df[df["p_num"] == "p12"].describe()

,bg-0:00,insulin-0:00,carbs-0:00,hr-0:00,steps-0:00,cals-0:00
count,25167.000000,25299.000000,572.000000,23822.000000,15076.000000,22568.000000
mean,7.852477,0.375746,53.353147,86.077705,49.680817,14.458452
std,2.828864,1.871322,34.212447,19.243867,70.360571,10.994446
min,2.800000,-0.307800,1.000000,49.700000,0.000000,1.760000
25%,6.000000,0.000000,25.000000,68.400000,0.000000,6.750000
50%,7.200000,0.075000,40.000000,84.300000,18.000000,8.100000
75%,9.000000,0.225000,75.000000,103.300000,77.000000,20.690000
max,22.200000,25.350000,143.000000,136.600000,567.000000,57.880000


In [119]:
df_test = load_data(dataset_type="test")
df_test[df_test["p_num"] == "p02"]
# https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ForecastingGridSearchCV.html

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:45,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00
244,p02_25872,p02,00:35:00,8.8,8.6,8.9,9.5,10.2,10.5,10.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,p02_25873,p02,09:50:00,9.2,9.3,8.9,8.3,7.9,7.7,7.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,p02_25874,p02,06:05:00,11.0,12.8,14.8,16.8,17.8,18.4,18.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,p02_25875,p02,03:15:00,10.9,10.2,9.5,9.2,8.9,8.5,8.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,p02_25876,p02,09:15:00,10.0,9.2,8.7,8.3,7.9,7.6,7.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,p02_26094,p02,23:45:00,14.1,14.0,13.8,13.2,12.9,12.8,13.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
467,p02_26095,p02,13:10:00,5.3,5.0,4.8,4.7,4.6,4.4,4.3,...,Walking,Walking,Walking,Walking,Walking,Walking,Walking,Walking,Walking,NaN
468,p02_26096,p02,15:05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
469,p02_26097,p02,19:25:00,5.7,5.7,6.1,6.7,7.3,8.2,9.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df_test["p_num"].unique()

array(['p01', 'p02', 'p04', 'p05', 'p06', 'p10', 'p11', 'p12', 'p15',
       'p16', 'p18', 'p19', 'p21', 'p22', 'p24'], dtype=object)

In [121]:
df["p_num"].unique()
# 6 patients for 5min model: 2,3,4,10,11,12
five_min_patients = ["p02", "p03", "p04", "p10", "p11", "p12"]

In [122]:
all_estimators("metric", as_dataframe=True)

,name,object
0,AUCalibration,<class 'sktime.performance_metrics.forecasting...
1,CRPS,<class 'sktime.performance_metrics.forecasting...
2,ConstraintViolation,<class 'sktime.performance_metrics.forecasting...
3,EmpiricalCoverage,<class 'sktime.performance_metrics.forecasting...
4,GeometricMeanAbsoluteError,<class 'sktime.performance_metrics.forecasting...
5,GeometricMeanRelativeAbsoluteError,<class 'sktime.performance_metrics.forecasting...
6,GeometricMeanRelativeSquaredError,<class 'sktime.performance_metrics.forecasting...
7,GeometricMeanSquaredError,<class 'sktime.performance_metrics.forecasting...
8,IntervalWidth,<class 'sktime.performance_metrics.forecasting...
9,LogLoss,<class 'sktime.performance_metrics.forecasting...


## 0: ForecastingGridSearchCV Example
https://www.sktime.net/en/stable/api_reference/auto_generated/sktime.forecasting.model_selection.ForecastingGridSearchCV.html


In [127]:
from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.split import ExpandingSlidingWindowSplitter
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import MeanSquaredError

y = df[df["p_num"] == "p02"]["bg-0:00"].dropna()
fh = [1, 2, 3, 4, 5, 6]
cv = ExpandingSlidingWindowSplitter(
    fh=fh, initial_window=12, step_length=12, max_expanding_window_length=24 * 12
)
forecaster = NaiveForecaster()
param_grid = {"strategy": ["last", "mean", "drift"]}
gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid=param_grid,
    cv=cv,
    scoring=MeanSquaredError(square_root=True),
)
gscv.fit(y)
y_pred = gscv.predict(fh)

/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:645: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:645: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/home/cjrisi/Projects/diabetes/nocturnal-hypo-gly-prob-forecast/.noctprob-venv/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:645: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.


In [128]:
gscv.cv_results_

,mean_test_MeanSquaredError,mean_fit_time,mean_pred_time,params,rank_test_MeanSquaredError
0,0.840197,0.001406,0.005974,{'strategy': 'last'},1.0
1,2.333300,0.001602,0.001949,{'strategy': 'mean'},3.0
2,0.847771,0.001538,0.002173,{'strategy': 'drift'},2.0


In [129]:
y_pred

34331    12.4
34332    12.4
34333    12.4
34334    12.4
34335    12.4
34336    12.4
Name: bg-0:00, dtype: float64

In [39]:
bg_value_vars = ["bg-" + str(hour) + ":" for hour in range(5, -1, -1)]
bg_value_vars
all_value_var_lists = []
var_strs = ["bg-", "insulin-", "carbs-", "hr-", "steps-", "cals-", "activity-"]
for var in var_strs:
    var_str = var
    var_list = []
    for hour in range(5, -1, -1):
        time_hour = var_str + str(hour)
        for minutes in range(55, -1, -5):
            if minutes < 10:
                time = time_hour + ":0" + str(minutes)
            else:
                time = time_hour + ":" + str(minutes)
            var_list.append(time)
    all_value_var_lists.append(var_list)

all_value_var_lists[1]

['insulin-5:55',
 'insulin-5:50',
 'insulin-5:45',
 'insulin-5:40',
 'insulin-5:35',
 'insulin-5:30',
 'insulin-5:25',
 'insulin-5:20',
 'insulin-5:15',
 'insulin-5:10',
 'insulin-5:05',
 'insulin-5:00',
 'insulin-4:55',
 'insulin-4:50',
 'insulin-4:45',
 'insulin-4:40',
 'insulin-4:35',
 'insulin-4:30',
 'insulin-4:25',
 'insulin-4:20',
 'insulin-4:15',
 'insulin-4:10',
 'insulin-4:05',
 'insulin-4:00',
 'insulin-3:55',
 'insulin-3:50',
 'insulin-3:45',
 'insulin-3:40',
 'insulin-3:35',
 'insulin-3:30',
 'insulin-3:25',
 'insulin-3:20',
 'insulin-3:15',
 'insulin-3:10',
 'insulin-3:05',
 'insulin-3:00',
 'insulin-2:55',
 'insulin-2:50',
 'insulin-2:45',
 'insulin-2:40',
 'insulin-2:35',
 'insulin-2:30',
 'insulin-2:25',
 'insulin-2:20',
 'insulin-2:15',
 'insulin-2:10',
 'insulin-2:05',
 'insulin-2:00',
 'insulin-1:55',
 'insulin-1:50',
 'insulin-1:45',
 'insulin-1:40',
 'insulin-1:35',
 'insulin-1:30',
 'insulin-1:25',
 'insulin-1:20',
 'insulin-1:15',
 'insulin-1:10',
 'insulin-1:05

In [5]:
import pandas as pd

example_prediction = df_test[df_test["id"] == "p02_25874"]
df_list = []
for val_var in all_value_var_lists:
    temp_df = pd.melt(
        example_prediction, id_vars=["id", "p_num", "time"], value_vars=val_var
    )
    temp_df = temp_df.rename(
        columns={
            "variable": val_var[0][:-4] + "time",
            "value": val_var[0][:-4] + "value",
        }
    )
    df_list.append(temp_df)

bg_df = df_list[0]
insulin_df = df_list[1]
carbs_df = df_list[2]
hr_df = df_list[3]
steps_df = df_list[4]
cals_df = df_list[5]
activity_df = df_list[6]
new_df = pd.concat(
    [
        bg_df,
        insulin_df.iloc[:, -1:],
        carbs_df.iloc[:, -1:],
        hr_df.iloc[:, -1:],
        steps_df.iloc[:, -1:],
        cals_df.iloc[:, -1:],
        activity_df.iloc[:, -1:],
    ],
    axis=1,
)

NameError: name 'df_test' is not defined

In [126]:
new_df

,id,p_num,time,bg-time,bg-value,insulin-value,carbs-value,hr-value,steps-value,cals-value,activity-value
0,p02_25874,p02,06:05:00,bg-0:00,7.9,0.0533,NaN,NaN,NaN,NaN,NaN
1,p02_25874,p02,06:05:00,bg-0:05,8.1,0.0500,NaN,NaN,NaN,NaN,NaN
2,p02_25874,p02,06:05:00,bg-0:10,8.3,0.0500,NaN,NaN,NaN,NaN,NaN
3,p02_25874,p02,06:05:00,bg-0:15,8.4,0.0748,NaN,NaN,NaN,NaN,NaN
4,p02_25874,p02,06:05:00,bg-0:20,8.6,0.0988,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
67,p02_25874,p02,06:05:00,bg-5:35,17.8,0.0500,NaN,NaN,NaN,NaN,NaN
68,p02_25874,p02,06:05:00,bg-5:40,16.8,0.0902,NaN,NaN,NaN,NaN,NaN
69,p02_25874,p02,06:05:00,bg-5:45,14.8,6.1609,65.0,NaN,NaN,NaN,NaN
70,p02_25874,p02,06:05:00,bg-5:50,12.8,0.2509,NaN,NaN,NaN,NaN,NaN


In [52]:
from sktime.benchmarking.forecasting import ForecastingBenchmark
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import MeanSquaredPercentageError
from sktime.split import ExpandingWindowSplitter
import pandas as pd

ImportError: cannot import name 'Longley' from 'sktime.datasets' (/Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/venv/lib/python3.9/site-packages/sktime/datasets/__init__.py)

In [59]:
from sktime.datasets import load_longley


def load_diabetes_data():
    df = load_data()
    df = clean_data(df)
    """Load diabetes multivariate time series data"""
    # Get data for a specific patient
    patient_data = df[df["p_num"] == "p01"].copy()

    # Set datetime index
    patient_data["time"] = pd.to_datetime("2023-01-01 " + patient_data["time"])
    patient_data = patient_data.set_index("time")

    # Select the features you want to include
    features_to_include = ["bg-0:00", "insulin-0:00", "carbs-0:00", "hr-0:00"]
    return patient_data[features_to_include]


y, X = load_longley()

# Update your dataset loaders
dataset_loaders = [load_longley]

# Add tasks to the benchmark

In [60]:
benchmark = ForecastingBenchmark()
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="mean", sp=12),
    estimator_id="NaiveForecaster-mean-v1",
)
benchmark.add_estimator(
    estimator=NaiveForecaster(strategy="last", sp=12),
    estimator_id="NaiveForecaster-last-v1",
)

In [65]:
cv_splitter = ExpandingWindowSplitter(
    initial_window=12,
    step_length=3,
    fh=3,
)

In [66]:
scorers = [MeanSquaredPercentageError()]

In [67]:
for dataset_loader in dataset_loaders:
    benchmark.add_task(
        dataset_loader,
        cv_splitter,
        scorers,
    )

/Users/tonychan/GlucoseML/nocturnal-hypo-gly-prob-forecast/venv/lib/python3.9/site-packages/sktime/benchmarking/benchmarks.py:131: UserWarning: Entity with ID [id=[dataset=load_longley]_[cv_splitter=ExpandingWindowSplitter]] already registered, but the ID is now being used to register another entity, OVERWRITING previous registered entity.
  self.validations.register(


In [68]:
results_df = benchmark.run("./forecasting_results.csv")
results_df.T

,0,1,2,3
validation_id,[dataset=load_airline]_[cv_splitter=ExpandingW...,[dataset=load_airline]_[cv_splitter=ExpandingW...,[dataset=load_longley]_[cv_splitter=ExpandingW...,[dataset=load_longley]_[cv_splitter=ExpandingW...
model_id,NaiveForecaster-last-v1,NaiveForecaster-mean-v1,NaiveForecaster-last-v1,NaiveForecaster-mean-v1
runtime_secs,0.292486,0.335346,0.039139,0.022603
MeanSquaredPercentageError_fold_0_test,0.024532,0.049681,0.017456,0.017456
MeanSquaredPercentageError_fold_1_test,0.020831,0.0737,NaN,NaN
MeanSquaredPercentageError_fold_2_test,0.001213,0.05352,NaN,NaN
MeanSquaredPercentageError_fold_3_test,0.01495,0.081063,NaN,NaN
MeanSquaredPercentageError_fold_4_test,0.031067,0.138163,NaN,NaN
MeanSquaredPercentageError_fold_5_test,0.008373,0.145125,NaN,NaN
MeanSquaredPercentageError_fold_6_test,0.007972,0.154337,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
